In [130]:
#Import Libraries
import pandas as pd
import openpyxl
from random import randint, shuffle

In [131]:
#Name of the Excel file and sheets to be used
file_name = r'SecretSantaDataSet.xlsx'
people_sheet = r'Pessoas'
incompatibles_sheet = r'Incompatibilidades'

In [132]:
#Import participants from excel and put them on a list
people = pd.read_excel(file_name, engine='openpyxl', sheet_name=people_sheet) #abre a sheet 'Pessoas'
people = people.iloc[:, 0] #lê a primeira coluna que é a que tem o nome das pessoas

In [133]:
#Import incompatibilities between participants and put them on a list of sets
incompatibilities = pd.read_excel(file_name, engine='openpyxl', sheet_name=incompatibles_sheet) # abre a sheet 'Incompatibilidades'
incompats = [] #cria uma lista vazia onde vai colocar todos os pares de incompatibilidades
for index, row in incompatibilities.iterrows(): #para cada linha da sheet existem nomes nas colunas A e B, o método iterrows (acho que vi online, mas não me lembro) itera sobre o dataframe  
    a_set = {row[0], row[1]} #cada linha do dataframe torna-se um set de 2 nomes, o da coluna A e o da coluna B, o set não é ordenado - as pessoas vão ser incompatíveis nos dois sentidos 
    incompats.append(a_set) #adiciona cada set de 2 pessoas incompatíveis à lista 

In [134]:
#FUNCTION FIND_INCOMPATIBILITIES
#Returns a set of people that a "name_set" cannot match with, based on the incompats list
#Returns  subset of the incompats list for a person
def find_incompatibilities(name_set, exclusion_list): 
    incompatibles = set() #criamos um set vazio (não tem ordem) onde iremos colocar todas as pessoas incompatíveis com a pessoa 'name_set', exemplo {Manel}
    for i in range(len(exclusion_list)): #iteramos sobre a lista de sets de incompats = [{Manel, António}, {Maria, Cristina}, {Zé, Teresa}]
        if name_set.issubset(exclusion_list[i]): #se o nome da pessoa 'name_set' estiver em algum dos sets da lista de incompats..., se {Manel} é subset de {Manel, António}
            incompatibles = incompatibles.union(exclusion_list[i]) #faz a união do set encontrado com o set de incompatíveis da pessoa, neste caso incompatibles = {Manel, António} (junta a própria pessoa, mas nunca repete pessoas porque é um set), se não encontrar nada incompatibles = {}
    return incompatibles

In [135]:
#FUNCTION FIND_POSSIBILITIES
#Returns the list of possible matches for a person, excluding themselves
#Returns the list of the difference between all participants and the incompatibilities for a person
def find_possibilities(name_set, receivers_set, exclusion_list): 
    possibilities_set = set() #criamos um set vazio onde iremos colocar todas as pessoas compatíveis com a pessoa 'name_set'
    possibilities_set = receivers_set.difference(name_set) #esse set = set de todos os participantes que ainda não foram escolhidos como recebedores de outra pessoa - a própria pessoa 'name_set'; este passo tem de estar aqui por causa das pessoas que não têm incompatibilidades
    possibilities_set = possibilities_set.difference(find_incompatibilities(name_set, exclusion_list)) #esse set = esse set - incomaptibilidades da pessoa 'name_set'
    return list(possibilities_set)

In [140]:
#Program
#Initial validations
end_program = False #variável de controlo
max_tries = 1000 #Número máximo de vezes que podemos tentar sortear até encontrarmos uma combinação válida
tries = 0 #contador de tentativas
if len(people) <2:
    print("Secret Santa needs at least 2 participants.")
    end_program = True

elif people.isnull().any:
    print("The participants list has null values.")
    end_Program = True

elif len(people) != len(set(people)):
    duplicates = [x for x in people if people.count(x)>1]
    print(f'There can\'t be participants with exactly the same name.\nThese are repeated: {set(duplicates)}')
    end_Program = True

#SortingHat
else: #Aqui já tudo foi validado, começa realmente o programa
    people = list(people) #converte dataframe em lista
    while tries < max_tries:
        shuffle(people) #reordenamento aleatório da lista de participantes, que faz com que a iteração da lista abaixo seja por uma ordem diferente a cada tentativa
        givers = people.copy() #criamos uma lista de dadores que é uma cópia dos participantes; não pode ser a própria lista de participantes porque vamos estar a alterar a lista de dadores e queremos que lhe seja feito reset a cada tentativa
        receivers = people.copy() #criamos uma lista de recebedores que é uma cópia dos participantes; não pode ser a própria lista de participantes porque vamos estar a alterar a lista de recebedores e queremos que lhe seja feito reset a cada tentativa
        matches = [] #lista com quem dá prenda a quem
        for p in people : #iteramos a lista de participantes, não a lista de dadores porque a lista de dadores é alterada a cada iteração
            #Gets a list of all possible receivers for a participant p
            all_possible_rec = find_possibilities({givers[0]}, set(receivers), incompats) #escrevemos lista de todos os nomes da lista de recebedores que podem receber prenda do dador que está na primeira posição da lista de dadores
            if len(all_possible_rec) == 0 : #se não encontra possibilidades, sai do ciclo for e volta a tentar o processo todo de novo (nova iteração no ciclo while)
                break
            #Selects a random match from all possible receivers
            match = all_possible_rec[randint(0, len(all_possible_rec)-1)] #atribui ao dador um recebedor aleatório do set que tinha determinado antes
            matches.append([givers[0], match]) #adiciona uma lista [dador, recebedor] à lista inteira de quem dá a prenda a quem
            givers.pop(0) #altera a lista de dadores, removendo o primeiro (o givers[0]), assim a lista é cortada à esquerda
            receivers.pop(receivers.index(match)) #açtera a lista de recebedores disponíveis removendo o que foi selecionado
        tries += 1 #depois de percorrer o ciclo todo para todas as pessoas da lista de participantes ou de ter saído porque não havia recebedores disponíveis para um dos participantes, aumenta o contador de tentativas em 1
        if len(matches) == len(people): #nova condição de saída do ciclo while: se todos os participantes têm alguém a quem dar prenda
            break

The participants list has null values.


In [139]:
#Converts results list into data frame and prints results
df_matches = []
if not end_program and tries != 0: #coisas que vamos fazer apenas se o programa correu e não falhou nas condições iniciais (entrou no else do Sorting hat)
        print(f'{tries} tries; {len(matches)} out of {len(people)} matches') #Número de tentativas; número de pessoas que conseguem dar prenda a outra e número total de participantes

        if tries == max_tries and len(matches) != len(people): # se nem todas as pessoas conseguiram dar prenda a outra, o programa termina
            print(f'Maybe try again next year with more participants and less restrictions.')
            end_Program = True

        else: #caso contrário, cria o dataframe com coluna de dadores e coluna de recebedores
            df_matches = pd.DataFrame(matches, columns=['Gift Giver', 'Gift Receiver'])
            print(df_matches)

In [138]:
#Writes results to a new sheet on Excel File
wb = openpyxl.load_workbook(file_name)
if 'Resultados' in wb.sheetnames: #qualquer que seja o cenário procura sempre a sheet 'Resultados' e apaga-a se existir
    wb.remove(wb['Resultados'])

if not end_program and tries != 0 and len(df_matches) != 0: #coisas que vamos fazer apenas se o programa correu e não falhou nas condições iniciais (entrou no else do Sorting hat)

        ws_new = wb.create_sheet('Resultados') #cria uma nova sheet de resultados no excel
        ws_new.append(list(df_matches.columns)) #diz que essa sheet vai ter duas colunas com os nomes dos headers do dataframe
        for i in range(len(df_matches)): #para cada linha do dataframe, escreve no excel
            ws_new.append(list(df_matches.loc[i, :]))

        ws_new.column_dimensions['A'].width = 55 #largura da coluna permite todos os nomes que temos no exemplo
        ws_new.column_dimensions['B'].width = 55

wb.save(file_name)
wb.close()
